 # Reinforcement Learning
 ## Final Project - The Lazy Gardener
 ### Contributors:
 - Nina Randorf
 - Remy Zhong
 - Arpit Parihar

 ### Importing modules

In [ ]:
import numpy as np
import pandas as pd
from TLG_env import garden_env

 ### Declare environment constants and transition matrices

In [ ]:
# Dictionaries to keep track of weather, bunny and actions
WEATHER = {0: 'Sunny', 1: 'Rainy', 2: 'Stormy'}
BUNNY = {0: 'No_Bunny', 1: 'Bunny'}
ACTIONS = {-1: 'Pump', 0: 'Nothing', 1: 'Water'}

# List of possible saturation states
STATES = list(range(-1, 6))

# Weather to weather transition matrix
P_WEATHER = pd.DataFrame(
    {
        0: [0.75, 0.3, 0.4],
        1: [0.2, 0.5, 0.4],
        2: [0.05, 0.2, 0.2]
    })

# Weather to bunny transition matrix
P_BUNNY = pd.DataFrame(
    {
        0: [0.2, 0.8],
        1: [0.6, 0.4],
        2: [0.9, 0.1]
    })

 ## Creating environment instance for Q-Learning algorithm

In [ ]:
QL_Agent = garden_env(WEATHER, BUNNY, ACTIONS, STATES, P_WEATHER, P_BUNNY)

 ### Declaring algorithm constants

In [ ]:
LEARNING_RATE = 0.1
DISCOUNT = 0.99
EPISODES = 10000
DAYS_TILL_HARVEST = 50
HARVEST_REWARD = 0

epsilon = 1
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES // 2
epsilon_decay_value = epsilon / (END_EPSILON_DECAYING - START_EPSILON_DECAYING)


 ### Algorithm loop

In [ ]:

q_table = np.random.uniform(low=-0.5, high=0.5, size=(len(QL_Agent.P_ind), len(QL_Agent.A)))

for episode in range(EPISODES):
    QL_Agent.reset_env()
    state = QL_Agent.state
    for day in range(DAYS_TILL_HARVEST):
        if np.random.random() > epsilon:
            action = np.argmax(q_table[QL_Agent.P_ind.index(state)])
        else:
            action = np.random.randint(0, len(QL_Agent.A))
        
        # rand_num = np.random.random()
        # P_cur = pd.DataFrame(P[action], index=P_ind, columns=P_ind)
        # state_new = pd.Series(P_ind)[P_cur.loc[state, :].cumsum().ge(rand_num).tolist()].iloc[0]
        QL_Agent.step(action)
        state_new = QL_Agent.state
        
        # reward = R[action, P_ind.index(state), P_ind.index(state_new)]
        reward = QL_Agent.reward if day < DAYS_TILL_HARVEST - 1 else QL_Agent.reward + HARVEST_REWARD
        
        max_future_q = np.max(q_table[QL_Agent.P_ind.index(state_new)])
        current_q = q_table[QL_Agent.P_ind.index(state), action]
        new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
        
        q_table[QL_Agent.P_ind.index(state), action] = new_q
        state = state_new
        if eval(state)[0] in [min(QL_Agent.S), max(QL_Agent.S)]:
            break
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
        epsilon -= epsilon_decay_value

In [ ]:
policy = pd.Series([QL_Agent.A[x] for x in q_table.argmax(axis=1) - 1], index=QL_Agent.P_ind)
# policy = pd.Series([x for x in q_table.argmax(axis=1) - 1], index=QL_Agent.P_ind)
Q_table_final = pd.DataFrame(q_table, columns=(QL_Agent.A.values()), index=QL_Agent.P_ind)
Q_table_final['Policy_QLearning'] = policy
# Q_table_final
# # %%
# # %%
# # %%
# QL_Agent.step(-1)
# QL_Agent.state
# QL_Agent.reward


In [ ]:
# Creating q vector
q = np.zeros((len(ACTIONS), len(QL_Agent.P_ind)), dtype=np.float64)

for i in ACTIONS:
    for j in range(len(QL_Agent.P_ind)):
        q[i, j] = np.dot(QL_Agent.P[i, j, :], QL_Agent.R[i, :])

T = 600
v = np.zeros((len(QL_Agent.P_ind), T + 1),dtype=np.float64)
d = np.zeros((len(QL_Agent.P_ind), T + 1),dtype=np.float64)

for t in range(1, T + 1):
    for i in range(len(QL_Agent.P_ind)):
        rhs = np.zeros(len(ACTIONS), dtype=np.float64)
        for j in ACTIONS:
            rhs[j] = q[j, i] + DISCOUNT*np.matmul(QL_Agent.P[j, i, :], v[:, t - 1])
        v[i, t] = max(rhs)
        d[i, t] = np.argmax(rhs)

v = pd.DataFrame(v.T, columns=QL_Agent.P_ind)
d = pd.DataFrame(d.T, columns=QL_Agent.P_ind)

In [ ]:
Q_table_final['Policy_QLearning'] = policy
policy = pd.Series([QL_Agent.A[x] for x in d.iloc[len(v)-1, :] - 1], index=QL_Agent.P_ind)
Q_table_final['Policy_VIter'] = policy
Q_table_final.loc[[eval(x)[0] not in [min(QL_Agent.S), max(QL_Agent.S)] for x in QL_Agent.P_ind], :]

In [ ]:

v = np.zeros((len(QL_Agent.P_ind), T + 1),dtype=np.float64)
d = np.zeros((len(QL_Agent.P_ind), T + 1),dtype=np.float64)

for t in range(1, T + 1):
    for i in range(len(QL_Agent.P_ind)):
        rhs = np.zeros(len(ACTIONS), dtype=np.float64)
        for j in ACTIONS:
            rhs[j] = q[j, i] + DISCOUNT*np.matmul(QL_Agent.P[j, i, :], v[:, t - 1])
        v[i, t] = max(rhs)
        d[i, t] = np.argmax(rhs)

    PP = np.array([QL_Agent.P[j, i, :] for (i, j) in enumerate(d[:, t].astype('int'))])
    A = np.concatenate((np.identity(len(QL_Agent.P_ind)) - PP, np.ones((len(QL_Agent.P_ind), 1))), axis=1)
    A = np.identity(len(QL_Agent.P_ind)) - PP
    A = np.delete(A, len(QL_Agent.P_ind)-1, 1)
    
    qq = np.array([q[j, i] for (i, j) in enumerate(d[:, t].astype('int'))])
    tmp = np.matmul(np.linalg.inv(A), qq)
    g = tmp[len(QL_Agent.P_ind)-1]
    tmp[len(QL_Agent.P_ind)-1] = 0
    v[:, t] = tmp.T
    
v = pd.DataFrame(v.T, columns=QL_Agent.P_ind)
d = pd.DataFrame(d.T, columns=QL_Agent.P_ind)
d.iloc[-1]

### DQN